In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from Base_de_datos.basededatos import BaseDeDatos
import re
import time
import requests
from selenium.webdriver.common.by import By
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.utf-8')


'es_ES.utf-8'

In [8]:
bd = BaseDeDatos()
bd.eliminarPlacasDeUnaTienda(tienda='Mercado Libre')

Se conecto a la base de datos


1

In [4]:
#Mercado Libre
#Estan las placas AMD y Nvidia en la misma pagina
url = 'https://www.mercadolibre.com.ar/'
bd = BaseDeDatos()
filtro_palabras = [
    "placa de video",
    "Tarjeta de video",
    "Envio gratis",
]

Se conecto a la base de datos


In [5]:
driver = webdriver.Chrome()
driver.get(url)
elemento = driver.find_element(By.ID, 'cb1-edit')
elemento.send_keys("Placa de video")
elemento.submit()
html = driver.page_source
time.sleep(5)
driver.quit()
html = BeautifulSoup(html, 'html.parser')
productos = html.find_all('div',class_='andes-card ui-search-result shops__cardStyles ui-search-result--core andes-card--flat andes-card--padding-16 andes-card--animated')
productos

[<div class="andes-card ui-search-result shops__cardStyles ui-search-result--core andes-card--flat andes-card--padding-16 andes-card--animated" id="react-aria-:Rgl9b9:"><div class="ui-search-result__image shops__picturesStyles"><section aria-label="Placa de video Nvidia MSI  Ventus GeForce RTX 20 Series RTX 2060 SUPER GEFORCE RTX 2060 SUPER VENTUS GP OC OC Edition 8GB" aria-roledescription="Carrusel" class="andes-carousel-snapped__container andes-carousel-snapped__container--full andes-carousel-snapped__container--with-controls andes-carousel-snapped__container--arrows-visible" id="react-aria-:R16gl9b9:"><div class="andes-carousel-snapped__header"></div><div class="andes-carousel-snapped__controls-wrapper"><button aria-label="Anterior" class="andes-carousel-snapped__control andes-carousel-snapped__control--previous andes-carousel-snapped__control--size-large andes-carousel-snapped__control--disabled" disabled="" type="button"><span aria-hidden="true" class="andes-carousel-snapped__cont

In [9]:
TIENDA =bd.idTienda("mercado libre")
driver = webdriver.Chrome()
for producto in productos:    
    try:
        nombre = producto.find('div', class_='ui-search-result__content').find('a', class_='ui-search-item__group__element shops__items-group-details ui-search-link')['title']
        
        if nombre.find('Rtx') != -1 or nombre.find('Nvidia') != -1 or nombre.find('Gtx') != -1 or nombre.find('Pny') != -1 or nombre.find("Gt") != -1:
            marca = 'Nvidia'
        else:
            marca = 'AMD'

        for palabra in filtro_palabras:
            if palabra.lower() in nombre.lower():
                nombre = nombre.lower().replace(palabra.lower(), "").capitalize()


        link = producto.find('div', class_='ui-search-result__content').find('a', class_='ui-search-item__group__element shops__items-group-details ui-search-link')['href']

        precio = producto.find('div', class_='ui-search-item__group ui-search-item__group--price ui-search-item__group--price-grid-container shops__items-group').find('div', class_='ui-search-price ui-search-price--size-medium shops__price').find('span', class_='andes-money-amount__fraction').text.strip()   
        precio = re.sub(r'[^0-9.]', '', precio)
        precio = precio.replace(".","")
        precio = int(precio)


        #####################
        driver.get(link)
        html = driver.page_source
        html = BeautifulSoup(html, 'html.parser')
        try:
            url_img = html.find('div', class_='ui-pdp--sticky-wrapper ui-pdp--sticky-wrapper-gallery').find('div', class_='ui-pdp-gallery__column').find('span', class_='ui-pdp-gallery__wrapper').find('figure', class_='ui-pdp-gallery__figure').find('img')['src']
        except Exception as e:
            print(e)
        ####################
        respuesta = requests.get(url_img)
        imagen = respuesta.content
       
        print(nombre[:14])
        print(link)
        print(TIENDA)
        print(precio)
        print(marca)
        print(imagen)
        bd.insertarPlaca(nombre=nombre,precio=precio,link=link,marca=marca,tienda=TIENDA,imagen=imagen)
        time.sleep(30)

        imagen = b''
        url_img = ''
        respuesta = b''


    except Exception as e:
        print(e)
driver.quit()

 nvidia msi  v
https://www.mercadolibre.com.ar/placa-de-video-nvidia-msi-ventus-geforce-rtx-20-series-rtx-2060-super-geforce-rtx-2060-super-ventus-gp-oc-oc-edition-8gb/p/MLA15758604?pdp_filters=category:MLA1658#searchVariation=MLA15758604&position=1&search_layout=grid&type=product&tracking_id=9bbeca84-ffa2-4e4d-b95a-be202ca36917
7
274900
Nvidia
b'RIFF\xfa>\x00\x00WEBPVP8 \xee>\x00\x00p\x11\x01\x9d\x01*\xf4\x01\x8d\x01>m4\x96H$"\xa2!%S\nx\x80\r\x89in\xe0\x1f\xf8!\x91\xdb"6>z\x9b\xe6\xfe\xcb\xc5Q\xd1\xbd~m\xb0\xf3!\xfbU\xebS\xe8#\xfd\x7f\xa6?Q\xe6\xf3\xc7\xee\xc6U\x97\xcf\xfc\x8byM\xfb\x9f\x0f|\xcf\xfb\xbb\xdd/\x8b\x8b\xf7\xfcw\xfa\x1eg\xff<\xfc\xb7\xfc\x8f\xf2\x9e\xd9\xff\xc3\xef\xcf\xe4\xd6\xa1\x7f\x92\xffK\xffQ\xf9\xb9\xe8\xeb\xf6\x83\xc0?y\xffc\xfbQ\xec)\xee/\xda\x7f\xdf\xff\x93\xf6\x15\xfb\xef\xfc\x9e\x8f\x7f\x0f\xfe\xaf\xfe\xaf\xb8\x17\xe6\x7f\xf7O>O\r\xff\xc9\xff\xd5\xfd\xa0\xf8\x05\xfe\x8f\xfd\xe3\xffg\xf9\xafx_\xf3?o=\x13\xfdU\xff\xd3\xfd\xa7\xc0o\xf4o\xee\xdf\xf7\x7f\xc6\xfbv\x

In [ ]:
bd.cerrarConexion()